# Coordinate Attention (Koordinat Dikkati) — Anlatım Notları

Bu notlar **Coordinate Attention (CA / CoordAtt)** mekanizmasını; **neden ortaya çıktığını**, **hangi problemi çözdüğünü**, **nasıl çalıştığını** ve **nerede daha verimli kullanıldığını** anlaşılır ama teknik doğrulukla özetler.

## 1) Coordinate Attention nedir?

**Coordinate Attention**, bir CNN ara-özellik haritası üzerinde **kanal (channel) ağırlıklandırmasını** yaparken aynı zamanda **konumsal (spatial) bilgiyi** korumayı hedefleyen bir dikkat (attention) bloğudur.

Klasik channel attention yaklaşımlarında (ör. SE/ECA gibi) özellik haritası genellikle **Global Average Pooling** ile tek bir vektöre indirgenir. Bu indirgeme, **"hangi kanal önemli?"** sorusunu cevaplar; ancak **"nerede önemli?"** bilgisi büyük ölçüde kaybolur.

Coordinate Attention’ın temel fikri:

- Uzamsal bilgiyi tamamen silen tek bir global havuzlama yerine,
- **Yükseklik (H) ve genişlik (W) eksenlerini ayrı ayrı** encode ederek,
- Kanal ağırlıklandırmasına **konum duyarlılığı (positional awareness)** kazandırmak.

## 2) Neden ortaya çıktı? (Motivasyon)

CNN tabanlı birçok modelde **attention** blokları performansı artırır; fakat iki sık görülen problem vardır:

1. **Global pooling kaynaklı konum kaybı**
   - SE (Squeeze-and-Excitation) gibi yapılar **H×W uzamını tek bir sayıya** indirger.
   - Bu, sınıflandırmada işe yarar; ancak **detection/segmentation** gibi görevlerde konum bilgisi kritik olduğundan, kazanç sınırlı kalabilir.

2. **2D spatial attention'ın maliyeti**
   - CBAM’in Spatial Attention kısmı gibi 2D maske üretimi (ör. 7×7 conv) bazı senaryolarda ek maliyet getirir.
   - Mobil/edge hedeflerde veya düşük gecikme istenen sistemlerde bu maliyet istenmeyebilir.

Coordinate Attention bu iki sorunu **dengeleyerek** çözmeyi hedefler:
- Konum bilgisini korur (global pooling gibi tamamen silmez).
- 2D maske üretmek yerine **1D eksen bazlı** temsillerle çalışarak maliyeti düşürür.

## 3) Hangi problemi çözer? (Çekirdek fikir)

Girdi özellik haritası:

\$
\mathbf{X} \in \mathbb{R}^{B \times C \times H \times W}
\$

Amaç, her konumda (h, w) için kanalları yeniden ölçeklemek:

\$
\mathbf{Y}_{c,h,w} = \mathbf{X}_{c,h,w} \cdot \alpha_{c,h} \cdot \beta_{c,w}
\$

Burada:
- \(\alpha\): **yükseklik yönü** boyunca öğrenilen dikkat katsayıları (H ekseni)
- \(\beta\): **genişlik yönü** boyunca öğrenilen dikkat katsayıları (W ekseni)

Bu çarpım, kanalı sadece "genel" biçimde değil, **eksene bağlı olarak** modüle eder. Yani CA, kanal dikkatini **konum ile bağlar**.

## 4) Nasıl çalışır? (Adım adım)

### 4.1) Eksen bazlı havuzlama (Coordinate Information Embedding)

SE’deki gibi tek bir global pooling yerine iki ayrı pooling yapılır:

- **Yükseklik yönünde havuzlama** (W boyunca ortalama):

\$
\mathbf{z}^{h}_{c}(h) = \frac{1}{W}\sum_{w=1}^{W} \mathbf{X}_{c,h,w}
\$
Sonuç boyutu: \(\mathbf{z}^h \in \mathbb{R}^{B \times C \times H \times 1}\)

- **Genişlik yönünde havuzlama** (H boyunca ortalama):

\$
\mathbf{z}^{w}_{c}(w) = \frac{1}{H}\sum_{h=1}^{H} \mathbf{X}_{c,h,w}
\$
Sonuç boyutu: \(\mathbf{z}^w \in \mathbb{R}^{B \times C \times 1 \times W}\)

Bu iki özet, uzamsal yapıyı tamamen yok etmez:
- \(\mathbf{z}^h\) yüksekliğe göre değişir
- \(\mathbf{z}^w\) genişliğe göre değişir

### 4.2) Paylaşımlı dönüşüm (Shared Transform)

Amaç: iki eksenden gelen bilgiyi **ortak bir gömleme uzayında** birleştirip kanallar arası ilişkiyi öğrenmek.

Genelde şu akış kullanılır:
1. \(\mathbf{z}^h\) ve \(\mathbf{z}^w\) uygun şekilde birleştirilir (concat).
2. Kanal sıkıştırma için \(1\times1\) konvolüsyon uygulanır:
\$
\mathbf{f} = \delta(\mathrm{BN}(\mathrm{Conv}_{1\times1}([\mathbf{z}^h; \mathbf{z}^w])))
\$
- \(\delta\): genelde **h-swish** (mobil uyumlu aktivasyon)
- Ara kanal sayısı çoğunlukla \(C/r\) (r: reduction oranı)

Bu aşama, SE’deki MLP benzeri yapının konvolüsyonla ve eksen temsilleriyle yapılmış halidir.

### 4.3) Dikkat haritalarının ayrıştırılması

Paylaşımlı \(\mathbf{f}\) tekrar iki kola ayrılır ve iki ayrı \(1\times1\) conv ile attention üretilir:

\$
\alpha = \sigma(\mathrm{Conv}_{1\times1}^{h}(\mathbf{f}^{h})) \in \mathbb{R}^{B \times C \times H \times 1}
\$

\$
\beta  = \sigma(\mathrm{Conv}_{1\times1}^{w}(\mathbf{f}^{w})) \in \mathbb{R}^{B \times C \times 1 \times W}
\$

- \(\sigma\): sigmoid

Son olarak çıktı:
\$
\mathbf{Y} = \mathbf{X} \odot \alpha \odot \beta
\$

Bu yapı, **2D spatial attention** üretmeden, iki eksen üzerinden “nerede önemli?” bilgisini taşır.

## 5) Neden iyi çalışır? (Sezgisel açıklama)

- **SE**: Kanalı global ölçekte ağırlıklandırır. Konum kaybolur.
- **CBAM-SA**: 2D maske ile “nerede”yi bulur ama kanal-uzay etkileşimi daha dolaylıdır ve ek maliyet getirir.
- **Coordinate Attention**:
  - Kanal önemini hesaplarken konumu tamamen yok etmez.
  - “Bu kanal **yükseklikte** nerede, **genişlikte** nerede daha aktif?” bilgisini taşır.
  - Özellikle yönsel/şerit benzeri yapılar (yol, kenar, yazı, insan silüeti gibi) için güçlü sinyal verir.

## 6) Hangi hedeflerde daha rahat kullanılır?

### 6.1) En çok fayda görülen görevler
- **Object Detection**: Kutuların yerini ve küçük objeleri yakalamada konum duyarlılığı değerlidir.
- **Segmentation**: Piksel seviyesinde kararlar konuma bağlıdır; CA bu bağlamda güçlüdür.
- **ReID / Fine-grained recognition**: İnsan/ürün gibi nesnelerde ayırt edici bölgeler yönsel dağılım gösterebilir.

### 6.2) Sınıflandırma (Classification)
- Sınıflandırmada da fayda sağlar, fakat bazen SE/ECA zaten yeterli olur.
- CA’nın avantajı, sınıflandırmada özellikle **arka plan karmaşası** ve **lokal ayırt edici bölgeler** (ör. kuş başı, araç farı) gibi durumlarda artar.

### 6.3) Mobil / düşük gecikme hedefleri
- CA tasarım olarak mobil dostudur (h-swish, 1×1 conv ağırlıklı).
- 2D spatial attention’a göre genellikle daha hafif bir alternatif sunar.

## 7) Parametre ve maliyet sezgisi

Bir CA bloğunda parametrelerin büyük kısmı **1×1 konvolüsyonlardan** gelir.

- Girdi kanal sayısı: \(C\)
- Reduction: \(r\) (ör. 16, 32)

Yaklaşık parametre ölçeği:
- Shared 1×1 conv: \(C \times (C/r)\)
- İki ayrı 1×1 conv: \((C/r)\times C\) + \((C/r)\times C\)

Toplam kabaca:
\$
\mathcal{O}\left(\frac{2C^2}{r}\right)
\$

Bu, 2D büyük kernel’li spatial attention (ör. 7×7 conv) ile kıyaslandığında genellikle daha kontrollü bir maliyet profili verir.

## 8) Pratik tasarım notları (uygulamada dikkat edilmesi gerekenler)

1. **Reduction (r) seçimi**
   - Çok küçük r (ör. 4): daha güçlü kapasite ama daha fazla maliyet
   - Çok büyük r (ör. 64): kapasite düşer, kazanç azalabilir
   - Mobilde genellikle 16/32 gibi değerler dengelidir.

2. **Aktivasyon**
   - CA literatürde sıkça **h-swish** kullanır; mobil mimarilerle uyumludur.
   - ReLU da kullanılabilir; ancak mobil hedefte h-swish tercih edilir.

3. **Yerleşim (block içinde nereye konur?)**
   - Bottleneck bloklarının sonunda veya residual eklemeden önce/sonra denenebilir.
   - Çok agresif bastırma oluyorsa (özellik std aşırı düşüyorsa), CA çıktısını residual ile dengelemek veya ölçekleme katsayısı eklemek iyi olabilir.

4. **CBAM ile birlikte kullanım**
   - CA zaten konum duyarlılığı getirdiği için, CBAM’in SA kısmını ağır tutmak çoğu zaman gereksizdir.
   - Daha sağlıklı hibrit: **CBAM-CA + CoordAtt** veya **CoordAtt + mikro SA (depthwise 3×3)**.

## 9) Coordinate Attention ile Coordinate Convolution aynı şey değil (kısa not)

Sıklıkla isim benzerliği karışır:

- **Coordinate Attention (CoordAtt)**: Attention bloğu. Amaç: feature’ları yeniden ağırlıklandırmak.
- **Coordinate Convolution (CoordConv)**: Konvolüsyon girişine koordinat kanalları (x, y, r gibi) ekleyerek modelin konumsal fonksiyonları daha kolay öğrenmesini sağlamak.

İkisi birlikte de kullanılabilir; ancak kavramsal olarak farklı bileşenlerdir.

## 10) Kod tarafı (çok kısa): PyTorch iskeleti

Aşağıdaki kod, mekanizmanın ana fikrini göstermek için minimal bir iskelettir. Projede kullanılacak üretim kodu; hata kontrolleri, cihaz/dtype uyumu ve optimizasyonlarla genişletilebilir.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HSwish(nn.Module):
    def forward(self, x):
        return x * F.relu6(x + 3.0, inplace=True) / 6.0

class CoordAtt(nn.Module):
    def __init__(self, in_channels: int, reduction: int = 32):
        super().__init__()
        mid = max(8, in_channels // reduction)  # pratikte alt sınır konur

        self.conv1 = nn.Conv2d(in_channels, mid, kernel_size=1, bias=False)
        self.bn1   = nn.BatchNorm2d(mid)
        self.act   = HSwish()

        self.conv_h = nn.Conv2d(mid, in_channels, kernel_size=1, bias=True)
        self.conv_w = nn.Conv2d(mid, in_channels, kernel_size=1, bias=True)

    def forward(self, x):
        b, c, h, w = x.shape

        # Height pooling: W boyunca ortalama -> (B, C, H, 1)
        x_h = x.mean(dim=3, keepdim=True)

        # Width pooling: H boyunca ortalama -> (B, C, 1, W)
        x_w = x.mean(dim=2, keepdim=True)
        x_w = x_w.permute(0, 1, 3, 2)  # (B, C, W, 1) gibi düşünmek için

        # Birleştir (H + W eksen bilgisi)
        y = torch.cat([x_h, x_w], dim=2)  # (B, C, H+W, 1)

        y = self.act(self.bn1(self.conv1(y)))

        # Tekrar ayır
        y_h, y_w = torch.split(y, [h, w], dim=2)

        y_w = y_w.permute(0, 1, 3, 2)  # (B, mid, 1, W)

        a_h = torch.sigmoid(self.conv_h(y_h))  # (B, C, H, 1)
        a_w = torch.sigmoid(self.conv_w(y_w))  # (B, C, 1, W)

        out = x * a_h * a_w
        return out

## 11) Kısa özet (akılda kalacak şekilde)

- **Coordinate Attention**, channel attention’a **konum duyarlılığı** katar.
- Bunu 2D spatial maskeyle değil; **H ve W eksenlerini ayrı encode ederek** yapar.
- **Detection/Segmentation** gibi konum hassas görevlerde genellikle daha belirgin kazanım sağlar.
- Mobil/düşük gecikme hedeflerde, klasik 2D spatial attention’a göre daha dengeli bir alternatiftir.